Import Libraries

In [156]:
import folium
import json
from shapely.geometry import Polygon
from folium.plugins import MarkerCluster
from shapely.geometry import shape, Point

Load Geojson data

In [157]:
with open('/content/F1.geojson', 'r') as f:
    geojson_data = json.load(f)

Create a map

In [158]:
m = folium.Map(location=[23.8130, 90.4304], zoom_start=16,min_zoom = 16,max_zoom=18)


Keep the original styling and colors of geojson file

In [159]:
# Function that returns the original style information for each feature
def get_style(feature):
    if feature['geometry']['type'] == 'Polygon':
        return {'fillColor': feature['properties']['fill'], 'color': feature['properties']['stroke'], 'weight': 1, 'fillOpacity': 0.8}
    elif feature['geometry']['type'] == 'Point':
        return {'fillColor': feature['properties']['marker-color'], 'color': feature['properties']['marker-color'], 'weight': 1, 'fillOpacity': 0.7}
    elif feature['geometry']['type'] == 'Circle':
        return {'fillColor': feature['properties']['fill'], 'color': feature['properties']['stroke'], 'weight': 1, 'fillOpacity': 0.7}


Styling of custom markers through code

In [160]:
# Marker for buildings
icon_css = """
    <style>
        .marker {
            width: 0;
            height: 0;
            border-left: 10px solid transparent;
            border-right: 10px solid transparent;
            border-bottom: 20px solid #f00;
            position: relative;
        }
        .marker::after {
            content: "";
            position: absolute;
            top: 20px;
            left: -10px;
            width: 0;
            height: 0;
            border-left: 10px solid transparent;
            border-right: 10px solid transparent;
            border-top: 20px solid #f00;
        }
    </style>
"""





# Define the size of the icon
#icon  = folium.Icon(icon='home', prefix='fa', color='black')


Loop through each feature and add it to the map

In [161]:
for feature in geojson_data['features']:
    if feature['geometry']['type'] == 'Polygon':
        # Create a GeoJSON layer for the polygon
        polygon = folium.GeoJson(feature,style_function=get_style)
        # Number of coordinates to identify circular plygon from the regular ones
        c_num = len(feature['geometry']['coordinates'][0])

        if c_num >= 50: #circular polygon
          # for Library building only
          if feature['properties']['name'] == 'Library':
            # Add a marker to the centroid of the polygon
            poly_coords = feature['geometry']['coordinates'][0]
            poly_shape = Polygon(poly_coords)
            centroid = poly_shape.centroid
            name = feature['properties']['name']
            marker = folium.Marker(location=[centroid.y, centroid.x], tooltip=name,icon=folium.Icon(icon='building', prefix='fa', color='black'))
            marker.add_to(polygon)
            polygon.add_to(m)
          else:
            # for other circular polygons
            polygon.add_to(m)
        else:
          # For normal polygons
          poly_coords = feature['geometry']['coordinates'][0]
          poly_shape = Polygon(poly_coords)
          centroid = poly_shape.centroid
          name = feature['properties']['name']
          marker = folium.Marker(location=[centroid.y, centroid.x], tooltip=name,icon=folium.Icon(icon='building', prefix='fa', color='black'))
          marker.add_to(polygon)
          polygon.add_to(m)
             
        
    # for adding the terminal markers to the map
    elif feature['geometry']['type'] == 'Point':
        name = feature['properties']['name']
        # Marker icon for terminals
        term_icon = folium.Icon(icon='bus', prefix='fa', color='red')
        marker = folium.Marker(location=[feature['geometry']['coordinates'][1], feature['geometry']['coordinates'][0]],
                               tooltip=name, icon=term_icon)
        
        # Add the marker to the map
        marker.add_to(m)
    else:
        # Add a GeoJSON layer for any other type of feature
        folium.GeoJson(feature).add_to(m)

In [162]:
display(m)

Pathway

In [163]:
testpath = '/content/T-1 to T-2.geojson'

In [164]:
def switchPosition(coordinate):
  temp = coordinate[0]
  coordinate[0] = coordinate[1]
  coordinate[1] = temp
  return coordinate

In [165]:
with open(testpath) as f:
  testWay = json.load(f)

for feature in testWay['features']:
    path = feature['geometry']['coordinates']
finalPath = list(map(switchPosition,path))
finalPath

[[23.81248814911102, 90.42448672167251],
 [23.81271677111272, 90.42557433814102],
 [23.813004451048002, 90.42892131258418]]

In [166]:
path = '/content/T-1 to T-2.geojson'
folium.plugins.AntPath(finalPath,color='#8B0000',opacity=0.9).add_to(m)
m